- Be able to grab the first hyper link of each wiki page
- Clicking on the first non-parenthesized, non-italicized link
- Ignoring external links, links to the current page, or red links (links to non-existent pages)
- Stopping when reaching "Philosophy", a page with no links or a page that does not exist, or when a loop occurs

if making a site: account for approximate names? to check for potential sites

check for loops
dead end sites


goals: a distribution?
a site for building a tree?
clustering / classification?



- Create a distribution and very population size?
- cluster and classify? (K-clusters and how to group -> most prominent links)
- is it possible to predict in what direction a page will branch?

for testing:
http://en.wikipedia.org/wiki/Special:Random

In [1]:
wiki_link = {
             "pokemon":"https://en.wikipedia.org/wiki/Pok%C3%A9mon",
             "tomcruise" :"https://en.wikipedia.org/wiki/Tom_Cruise",
             "korean":"https://en.wikipedia.org/wiki/Korean_Wikipedia",
             "popcorn":"https://en.wikipedia.org/wiki/Popcorn",
             "religion":'https://en.wikipedia.org/wiki/Importance_of_religion_by_country',
             "ygo":"https://en.wikipedia.org/wiki/Yu-Gi-Oh!",
             "frank":"https://en.wikipedia.org/wiki/Frankenstein",
             "foodwar":"https://en.wikipedia.org/wiki/Food_Wars!:_Shokugeki_no_Soma",
             "disneystudio":"https://en.wikipedia.org/wiki/Walt_Disney_Pictures",
             "knowledge":"https://en.wikipedia.org/wiki/Knowledge",
             "phylotype":"https://en.wikipedia.org/wiki/Phylotype"
            }

In [2]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import urllib.request
import urllib

from PIL import Image
from io import BytesIO

In [39]:
class Page:
    
    wiki = 'https://en.wikipedia.org'
    
    end = "https://en.wikipedia.org/wiki/Philosophy"
    
    branches = 0
    
    def __init__(self, url):
        self.url = url
        self.soup = BeautifulSoup(urlopen(self.url))
        self.title = self.soup.title.string
        
        try:
            self.image = Image.open(BytesIO(urlopen(self.soup.find_all("meta")[-1]["content"]).read()))
        except:
            self.image = None
            
    
    def remove_parentheses(self, body_text):
        """converts body of text from BeautifulSoup Object into String
        then takes the string and filters out all parenthesized strings/ links
        and returns the filtered obj reconverted into BeautifulSoup obj
        """
        
        out = []
        paren = False
        for i in str(body_text).split(" "):
    
            if "(" in i and "href" not in i:
                paren = True
            if ")" in i:
                paren = False
    
            if not paren:
                if ")" not in i or "href" in i:
                    out.append(i)

        return BeautifulSoup(' '.join(out))
            
            
    def all_links(self):
        """check_next takes a beautifulsoup object.
        returns all links found in bodies of text (tags "p")
        the result is a list wiki links in each body of text
        """
        
        res = []
        for tags in self.soup.find("body").find_all("p"):
            tags = self.remove_parentheses(tags)
            if tags.find("a") == None:
                pass
            else:
                for links in tags.find_all('a'):
                    try:
                        if "/wiki/" in links["href"]:
                            res.append(links["href"])
                    except:
                        pass
        return res
    
    
    def get_nexturl(self):
        for link in self.all_links():
            return self.wiki + link
                
    
    def crawl(self):
        """Crawls from current wiki page to next.
        
        Should check for cycles and break if one occurs 
        """
        
        #should Page class be nodes with .next (pointers?)
        
        visited = []
        self.branches = 0
        
        print(self.title + ":", self.url)
        visited.append(self.url)
        
        next_link = self.get_nexturl()
        
        while next_link != self.end:
            self.branches += 1
            
            temp_page = Page(next_link)
            visited.append(next_link)
            
            print(temp_page.title + ":", temp_page.url)
            
            next_link = temp_page.get_nexturl()
            
            if next_link in visited:
                raise Exception("Cycle was discovered. "+ next_link)
            
        self.branches += 1
        print("\033[1m Philosophy Reached! \033[0m", self.wiki+next_link)
    


In [19]:
pokemon = Page("https://en.wikipedia.org/wiki/Pok%C3%A9mon")
pokemon.crawl()
#pokemon.all_links()

Pokémon - Wikipedia: https://en.wikipedia.org/wiki/Pok%C3%A9mon
Multimedia franchise - Wikipedia: https://en.wikipedia.org/wiki/Media_franchise
Media (communication) - Wikipedia: https://en.wikipedia.org/wiki/Media_(communication)
Communication - Wikipedia: https://en.wikipedia.org/wiki/Communication
Meaning (linguistics) - Wikipedia: https://en.wikipedia.org/wiki/Meaning_(linguistics)
Linguistics - Wikipedia: https://en.wikipedia.org/wiki/Linguistics
Science - Wikipedia: https://en.wikipedia.org/wiki/Science
Knowledge - Wikipedia: https://en.wikipedia.org/wiki/Knowledge
Experience - Wikipedia: https://en.wikipedia.org/wiki/Experience
 Philosophy Reached!  https://en.wikipedia.org/wiki/Philosophy


In [35]:
exp = Page("https://en.wikipedia.org/wiki/Experience")
exp.crawl()
exp.branches

Experience - Wikipedia: https://en.wikipedia.org/wiki/Experience
 Philosophy Reached!  https://en.wikipedia.orghttps://en.wikipedia.org/wiki/Philosophy


1

In [16]:
foodwar = Page("https://en.wikipedia.org/wiki/Food_Wars!:_Shokugeki_no_Soma")
foodwar.crawl()

Food Wars!: Shokugeki no Soma - Wikipedia: https://en.wikipedia.org/wiki/Food_Wars!:_Shokugeki_no_Soma
Shōnen manga - Wikipedia: https://en.wikipedia.org/wiki/Sh%C5%8Dnen_manga
Manga - Wikipedia: https://en.wikipedia.org/wiki/Manga
Comics - Wikipedia: https://en.wikipedia.org/wiki/Comics
Media (communication) - Wikipedia: https://en.wikipedia.org/wiki/Media_(communication)
Communication - Wikipedia: https://en.wikipedia.org/wiki/Communication
Meaning (linguistics) - Wikipedia: https://en.wikipedia.org/wiki/Meaning_(linguistics)
Linguistics - Wikipedia: https://en.wikipedia.org/wiki/Linguistics
Science - Wikipedia: https://en.wikipedia.org/wiki/Science
Knowledge - Wikipedia: https://en.wikipedia.org/wiki/Knowledge
Experience - Wikipedia: https://en.wikipedia.org/wiki/Experience
 Philosophy Reached!  https://en.wikipedia.org/wiki/Philosophy


In [40]:
#cycles and check for cycles
tomcruise = Page("https://en.wikipedia.org/wiki/Tom_Cruise")
tomcruise.crawl()

Tom Cruise - Wikipedia: https://en.wikipedia.org/wiki/Tom_Cruise
List of awards and nominations received by Tom Cruise - Wikipedia: https://en.wikipedia.org/wiki/List_of_awards_and_nominations_received_by_Tom_Cruise


Exception: Cycle was discovered. https://en.wikipedia.org/wiki/Tom_Cruise

In [13]:
religion = Page(wiki_link["religion"])
religion.crawl()
#after country should be birth? why did we go to sovereign state?

Importance of religion by country - Wikipedia: https://en.wikipedia.org/wiki/Importance_of_religion_by_country
Gallup (company) - Wikipedia: https://en.wikipedia.org/wiki/Gallup_Poll
Washington, D.C. - Wikipedia: https://en.wikipedia.org/wiki/Washington,_D.C.
Capital city - Wikipedia: https://en.wikipedia.org/wiki/Capital_city
Municipality - Wikipedia: https://en.wikipedia.org/wiki/Municipality
Administrative division - Wikipedia: https://en.wikipedia.org/wiki/Administrative_division
Country - Wikipedia: https://en.wikipedia.org/wiki/Country
Birth - Wikipedia: https://en.wikipedia.org/wiki/Birth
Offspring - Wikipedia: https://en.wikipedia.org/wiki/Offspring
Biology - Wikipedia: https://en.wikipedia.org/wiki/Biology
Natural science - Wikipedia: https://en.wikipedia.org/wiki/Natural_science
Branches of science - Wikipedia: https://en.wikipedia.org/wiki/Branches_of_science
Science - Wikipedia: https://en.wikipedia.org/wiki/Science
Knowledge - Wikipedia: https://en.wikipedia.org/wiki/Knowl